Questão 2 - Considere os projetos a seguir e faça a escolha de dois deles para você (ou sua
equipe) desenvolver completamente uma solução, usando um processo de
desenvolvimento com a técnica de programação em pares, onde o outro par é um
LLM (de sua livre escolha), gerando sistemas de software que funcionem. No
desenvolvimento de cada projeto escolhido, realize o seguinte:

Projeto 2: Suponha que um gerente de um determinado banco, contratou você para
desenvolver um sistema de Inteligência Artiﬁcial, especiﬁcamente seguindo uma
abordagem de sistema baseado em conhecimento (conforme arquitetura conceitual
apresentada durante a disciplina), com regras fuzzy para cumprir a tarefa de decidir sobre o
risco de emprestar dinheiro, a cada demanda de seus clientes, tendo ainda a característica
de justiﬁcar cada decisão fornecida. Considere 3 categorias de risco: Alto, Moderado e
Baixo.
As funcionalidades exigidas, incluem:

a) Base de conhecimento a ser gerada automaticamente, via técnica de aprendizado de
máquina, considerando, por exemplo, a Base de Dados disponível em
https://www.kaggle.com/datasets/marcelotc/german-credit-risk,

b) Engenho de Inferência,

d) Explicabilidade das decisões tomadas pelo sistema,

e) Interface com suporte a linguagem natural via Chatbot, tal como pedido na lista 2.

In [ ]:
# Instalação dos Pacotes
!pip install scikit-fuzzy scikit-learn nltk gradio

In [ ]:
# Imports das Bibliotecas
import numpy as np
import pandas as pd
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import skfuzzy as fuzz
import skfuzzy.control as ctrl
import gradio as gr
import nltk
import re

In [ ]:
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# 1. Carregar base de dados
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data"
columns = [
    'Status', 'Duration', 'CreditHistory', 'Purpose', 'CreditAmount', 'Savings',
    'EmploymentSince', 'InstallmentRate', 'PersonalStatusSex', 'OtherDebtors',
    'ResidenceSince', 'Property', 'Age', 'OtherInstallmentPlans', 'Housing',
    'ExistingCredits', 'Job', 'LiablePeople', 'Telephone', 'ForeignWorker', 'Risk'
]
df = pd.read_csv(url, sep=' ', names=columns)

# Simplificação para variáveis contínuas
df['Risk'] = df['Risk'].map({1: 0, 2: 1})  # 0: bom, 1: ruim
df['Age'] = df['Age'].astype(int)
df['CreditAmount'] = df['CreditAmount'].astype(int)
df['Duration'] = df['Duration'].astype(int)

# Selecionar variáveis contínuas
data = df[['Duration', 'CreditAmount', 'Age', 'Risk']]

# 2. Aprendizado para gerar regras fuzzy
X = data[['Duration', 'CreditAmount', 'Age']]
y = data['Risk']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar uma árvore para identificar padrões
clf = DecisionTreeClassifier(max_depth=3)
clf.fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

# 3. Definir sistema fuzzy baseado nas variáveis
duration = ctrl.Antecedent(np.arange(0, 73, 1), 'duration')
credit = ctrl.Antecedent(np.arange(0, 20001, 100), 'credit')
age = ctrl.Antecedent(np.arange(18, 76, 1), 'age')
risk = ctrl.Consequent(np.arange(0, 11, 1), 'risk')

# Membership functions
duration.automf(3)
credit.automf(3)
age.automf(3)

risk['low'] = fuzz.trimf(risk.universe, [0, 0, 4])
risk['medium'] = fuzz.trimf(risk.universe, [2, 5, 8])
risk['high'] = fuzz.trimf(risk.universe, [6, 10, 10])

# 4. Regras fuzzy baseadas no modelo treinado
rule1 = ctrl.Rule(duration['poor'] | credit['poor'] | age['poor'], risk['high'])
rule2 = ctrl.Rule(duration['average'] & credit['average'] & age['average'], risk['medium'])
rule3 = ctrl.Rule(duration['good'] & credit['good'] & age['good'], risk['low'])

# 5. Engenho de inferência
risk_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])
risk_simulation = ctrl.ControlSystemSimulation(risk_ctrl)

# 6. Função para justificativa
def classify_credit(duration_val, credit_val, age_val):
    risk_simulation.input['duration'] = duration_val
    risk_simulation.input['credit'] = credit_val
    risk_simulation.input['age'] = age_val
    risk_simulation.compute()

    fuzzy_score = risk_simulation.output['risk']
    if fuzzy_score <= 3:
        level = 'Baixo'
    elif fuzzy_score <= 6:
        level = 'Moderado'
    else:
        level = 'Alto'

    explanation = f"""
    Justificativa da decisão:
    - Duração do empréstimo: {duration_val}
    - Valor do crédito: {credit_val}
    - Idade do cliente: {age_val}
    → Resultado fuzzy: {fuzzy_score:.2f}
    → Nível de risco: {level}
    """

    return level, explanation

# 7. Chatbot com linguagem natural
def chatbot_response(message):
    try:
        # Extrai todos os números da mensagem (espera-se: duração, crédito, idade)
        numeros = list(map(int, re.findall(r'\d+', message)))

        if len(numeros) != 3:
            return "Erro: Forneça exatamente 3 números (duração, crédito, idade)."

        duration_val, credit_val, age_val = numeros
        level, explanation = classify_credit(duration_val, credit_val, age_val)
        return f"Risco estimado: {level}\n\n{explanation}"

    except Exception as e:
        return f"Ocorreu um erro: {e}"

# 8. Interface Gradio
gr.Interface(fn=chatbot_response,
             inputs=gr.Textbox(label="Digite a consulta (ex: cliente com 24 meses, 5000 de crédito, 35 anos)"),
             outputs="text",
             title="Sistema Fuzzy de Risco de Crédito",
             description="Forneça duração, valor do crédito e idade. Exemplo: 'Cliente com 12 meses, 3000 de crédito, 45 anos.'"
            ).launch()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


              precision    recall  f1-score   support

           0       0.73      0.94      0.82       141
           1       0.50      0.15      0.23        59

    accuracy                           0.70       200
   macro avg       0.61      0.54      0.53       200
weighted avg       0.66      0.70      0.65       200

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9f8e5a49f9c5b8ff50.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Projeto 5: Elabore e Implemente uma solução via um sistema de software baseado em
agente conversaional que usa LLM e RAG para resolver um problema cuja entrada seja um
documento em PDF, daí o agente vai aprender sobre seu conteúdo e então poder conversar
com um usuário sobre o conteúdo do arquivo, por exemplo, respondendo perguntas.
Apresente o detalhamento das fases, seguindo o que já foi pedido na lista 2.

In [ ]:
## 1. Instalação e Imports
!pip install pymupdf faiss-cpu sentence-transformers openai google-generativeai ipywidgets -q

import fitz  # PyMuPDF
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from google.colab import files
import os
import openai
import google.generativeai as genai
import ipywidgets as widgets
from IPython.display import display
from google.colab import files


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 729.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.

In [ ]:
## 2. Upload e Processamento do PDF
uploaded = files.upload()
filename = next(iter(uploaded))

# Extração de texto
doc = fitz.open(filename)
full_text = ""
for page in doc:
    full_text += page.get_text()

# Quebra do texto em chunks de no máximo 500 caracteres
def chunk_text(text, max_len=500):
    sentences = text.split(". ")
    chunks, chunk = [], ""
    for sent in sentences:
        if len(chunk) + len(sent) <= max_len:
            chunk += sent + ". "
        else:
            chunks.append(chunk.strip())
            chunk = sent + ". "
    if chunk:
        chunks.append(chunk.strip())
    return chunks

chunks = chunk_text(full_text)

TypeError: 'NoneType' object is not subscriptable

In [ ]:
## 3. Geração e Armazenamento dos Embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(chunks)

# 5. Indexação com FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

# 6. Função para recuperação de contexto
def retrieve_context(query, k=3):
    query_embedding = model.encode([query])
    D, I = index.search(np.array(query_embedding), k)
    return "\n".join([chunks[i] for i in I[0]])

# 7. Configuração das APIs
openai.api_key = "xxxxxx"
genai.configure(api_key="xxxxxx")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

NameError: name 'chunks' is not defined

In [ ]:
## 4. Chat com RAG usando LLM 1 (ex: OpenAI GPT-3.5)
def answer_with_openai(query):
    context = retrieve_context(query)
    prompt = f"Com base no seguinte conteúdo:\n{context}\nResponda: {query}"
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response['choices'][0]['message']['content']

In [ ]:
## 5. Chat com RAG usando LLM 2 (ex: Gemini Pro)
def answer_with_gemini(query):
    context = retrieve_context(query)
    prompt = f"Com base no seguinte conteúdo:\n{context}\nResponda: {query}"
    model = genai.GenerativeModel("gemini-pro")
    response = model.generate_content(prompt)
    return response.text

In [ ]:
## 6. Comparação e Registro das Interações
llm_selector = widgets.Dropdown(options=["OpenAI", "Gemini"], description='LLM:')
question_input = widgets.Text(placeholder='Digite sua pergunta aqui...')
output_area = widgets.Output()

def on_submit(_):
    with output_area:
        output_area.clear_output()
        query = question_input.value
        if llm_selector.value == "OpenAI":
            print("Resposta (GPT-3.5):")
            print(answer_with_openai(query))
        else:
            print("Resposta (Gemini):")
            print(answer_with_gemini(query))

button = widgets.Button(description="Perguntar")
button.on_click(on_submit)

display(llm_selector, question_input, button, output_area)

Dropdown(description='LLM:', options=('OpenAI', 'Gemini'), value='OpenAI')

Text(value='', placeholder='Digite sua pergunta aqui...')

Button(description='Perguntar', style=ButtonStyle())

Output()